In [ ]:
import numpy as np
import pandas as pd
import sys,os
import pickle
import hicstraw
from joblib import Parallel, delayed

In [ ]:
def zoomdata(row,hic,cord1=1,cord2=2,cord3=6,cord4=7,diagonal=False): 
    if diagonal==True:
        return np.diag(hic.getRecordsAsMatrix(row[cord1],row[cord2],row[cord3],row[cord4])).mean()  
    else:
        g=np.array(sorted(hic.getRecordsAsMatrix(row[cord1],row[cord2],row[cord3],row[cord4]).flatten()))
        if len(g)<4:
            return g.sum()
        else:
            return g[-3:].sum()
    
def zoombgneighbor(row,hic,cord1=1,cord2=2,cord3=6,cord4=7,diagonal=False,exclude=5000,include=50000): #bg estimator
        n11=row[cord1]-include
        n12=row[cord1]-exclude
        n21=row[cord2]+exclude
        n22=row[cord2]+include
        n31=row[cord3]-include
        n32=row[cord3]-exclude
        n41=row[cord4]+exclude
        n42=row[cord4]+include
        if row[cord1]<row[cord4]:
            res1=hic.getRecordsAsMatrix(n11,n12,n41,n42).mean()
            return res1
        else:
            res2=hic.getRecordsAsMatrix(n21,n22,n31,n32).mean()
            return res2

try:
    candf
except NameError:
    candf = [] 
    
def plf7(filename,df=candf):
    print(filename)
    hic = hicstraw.HiCFile(folder+filename)
    results=[]
    print(df)
    for i in df[0].unique():
        key=str(i)
        hic1=hic.getMatrixZoomData(key,key, "observed", "KR", "BP", 1000)
        sub=df[df[0]==i]
        results.append(sub.apply(zoomdata,hic=hic1,cord3=6,cord4=7,axis=1))
    results=pd.concat(results)
    print('ready')
    return(filename.split(".")[0],results)

def pbk7(filename,df=candf):
    print(filename)
    hic = hicstraw.HiCFile(folder+filename)
    results=[]
    print(df)
    for i in df[0].unique():
        key=str(i)
        hic1=hic.getMatrixZoomData(key,key, "observed", "KR", "BP", 1000)
        sub=df[df[0]==i]
        results.append(sub.apply(zoombgneighbor,hic=hic1,cord3=6,cord4=7,axis=1))
    results=pd.concat(results)
    print('ready')
    return(filename.split(".")[0],results)


In [ ]:
#all atac+TSS peak pairs within 200 kb

!module load bedtools2
!bedtools window -a atac_final.bed -b atac_final.bed -w 200000 |awk '$7>$3' > atac_finall_200k_full.bedpe

In [ ]:
candidate=pd.read_csv('atac_finall_200k_full.bedpe',header=None,sep="\t")
hic = hicstraw.HiCFile('all.hg38.nodups.pairs.hic')

hics=os.listdir('hics/')
hics=sorted([i for i in hics if 'all' not in i and '.hic' in i])

In [ ]:
#selecting candidate interaction pairs with some contact frequency

resultsk=[]
results=[]
for i in candidate[0].unique():
    key=str(i)
    data1=hic.getMatrixZoomData(key,key, "observed", "KR", "BP", 1000)
    data2=hic.getMatrixZoomData(key,key, "observed", "NONE", "BP", 1000)
    sub=candidate[candidate[0]==i]
    resultsk.append(sub.apply(zoomdata,hic=data1,axis=1))
    results.append(sub.apply(zoomdata,hic=data2,axis=1))
    
candidate['KR']=pd.concat(resultsk)
candidate['NN']=pd.concat(results)
candf=candidate[np.logical_and(candidate['NN']>=5,candidate['KR']>=1)]


In [ ]:
#raw balanced contact frequency
lf=Parallel(n_jobs=16)(delayed(plf7)(i) for i in hics) #[j for j in orders])
lfk=pd.DataFrame(dict(lf))

In [ ]:
#background 
bn = Parallel(n_jobs=16)(delayed(pbk7)(i) for i in hics)
bg = pd.DataFrame(dict(bn))

In [ ]:
#normalized connectivity 
lff=lfk.divide(bg.mean())*(bg.mean().mean())